In [1]:
import pandas as pd
#Load my dataset with an updated date format and lets get analyzing
df = pd.read_csv('C:/Users/srsch/OneDrive/Desktop/merged_steel_weather_data_with_datetime.csv')

In [2]:
import pandas as pd

# Load dataset
file_path = r'C:/Users/srsch/OneDrive/Desktop/merged_steel_weather_data_with_datetime.csv'
try:
    df = pd.read_csv(file_path)
    print(f"Loaded {file_path}")
    print("Shape:", df.shape)
    print("Columns:", df.columns.tolist())
    print("Sample data (first 5 rows):")
    print(df.head())
    print("Missing values:", df.isnull().sum().to_dict())
except FileNotFoundError:
    print(f"{file_path} not found! Please confirm the file path.")
    exit()

Loaded C:/Users/srsch/OneDrive/Desktop/merged_steel_weather_data_with_datetime.csv
Shape: (35040, 20)
Columns: ['date', 'Usage_kWh', 'Lagging_Current_Reactive.Power_kVarh', 'Leading_Current_Reactive_Power_kVarh', 'CO2(tCO2)', 'Lagging_Current_Power_Factor', 'Leading_Current_Power_Factor', 'NSM', 'WeekStatus', 'Day_of_week', 'Load_Type', 'date_only', 'time_only', 'time_numeric_dec', 'time_seconds_since_midnight', 'temperature_2m (°C)', 'relative_humidity_2m (%)', 'dew_point_2m (°C)', 'precipitation (mm)', 'Load_Type_Rank']
Sample data (first 5 rows):
                  date  Usage_kWh  Lagging_Current_Reactive.Power_kVarh  \
0  2018-01-01 00:15:00       3.17                                  2.95   
1  2018-01-01 00:30:00       4.00                                  4.46   
2  2018-01-01 00:45:00       3.24                                  3.28   
3  2018-01-01 01:00:00       3.31                                  3.56   
4  2018-01-01 01:15:00       3.82                                  4.

In [4]:
import pandas as pd
import numpy as np

# Load dataset
file_path = r'C:/Users/srsch/OneDrive/Desktop/merged_steel_weather_data_with_datetime.csv'
try:
    df = pd.read_csv(file_path)
    print(f"Loaded {file_path}, shape: {df.shape}")
except FileNotFoundError:
    print(f"{file_path} not found!")
    exit()

# Verify required columns
required_cols = ['CO2(tCO2)', 'Usage_kWh', 'date', 'temperature_2m (°C)', 'relative_humidity_2m (%)', 'Day_of_week']
print("Input columns:", df.columns.tolist())
missing_cols = [col for col in required_cols if col not in df.columns]
if missing_cols:
    print("Missing columns:", missing_cols)
    exit()

# Create humidity_binned
df['humidity_binned'] = pd.qcut(df['relative_humidity_2m (%)'], q=3, labels=['Low', 'Medium', 'High'])
print("Humidity bins:", df['humidity_binned'].unique())

# Create CO2_imputed
df['CO2_imputed'] = df.groupby('humidity_binned')['CO2(tCO2)'].transform(
    lambda x: x.replace(0, x[x != 0].mean() if x[x != 0].mean() else x)
)

# Remove outliers
Q1 = df['CO2_imputed'].quantile(0.25)
Q3 = df['CO2_imputed'].quantile(0.75)
IQR = Q3 - Q1
df_no_outliers = df[(df['CO2_imputed'] >= Q1 - 1.5 * IQR) & (df['CO2_imputed'] <= Q3 + 1.5 * IQR)].copy()

# Add time-based and lag features
df_no_outliers['date_time'] = pd.to_datetime(df_no_outliers['date'])
df_no_outliers['hour_of_day'] = df_no_outliers['date_time'].dt.hour
df_no_outliers['is_weekend'] = df_no_outliers['Day_of_week'].isin(['Saturday', 'Sunday']).astype(int)
for lag in range(1, 5):
    df_no_outliers[f'CO2_lag_{lag}'] = df_no_outliers['CO2_imputed'].shift(lag)
    df_no_outliers[f'kWh_lag_{lag}'] = df_no_outliers['Usage_kWh'].shift(lag)

# Add additional features (from truncated file)
df_no_outliers['temp_humid_interaction'] = df_no_outliers['temperature_2m (°C)'] * df_no_outliers['relative_humidity_2m (%)']
df_no_outliers['log_kWh'] = np.log1p(df_no_outliers['Usage_kWh'])
df_no_outliers['log_CO2'] = np.log1p(df_no_outliers['CO2(tCO2)'])
df_no_outliers['log_humidity'] = np.log1p(df_no_outliers['relative_humidity_2m (%)'])
df_no_outliers['yj_CO2'] = np.log1p(df_no_outliers['CO2(tCO2)'])  # Placeholder, adjust if Yeo-Johnson
df_no_outliers['yj_kWh'] = np.log1p(df_no_outliers['Usage_kWh'])
df_no_outliers['qt_CO2'] = df_no_outliers['CO2(tCO2)'].rank(pct=True)  # Quantile transform placeholder
df_no_outliers['qt_kWh'] = df_no_outliers['Usage_kWh'].rank(pct=True)

# Drop rows with NaN lags
df_no_outliers = df_no_outliers.dropna()

# Verify
print("Shape:", df_no_outliers.shape)  # Expect ~25,241
print("Columns:", df_no_outliers.columns.tolist())
print("Sample CO2_imputed:", df_no_outliers['CO2_imputed'].head().tolist())
print("Humidity bins:", df_no_outliers['humidity_binned'].unique())
print("Median CO2_imputed by bin:")
medians = df_no_outliers.groupby('humidity_binned')['CO2_imputed'].median()
print(medians)
print("Median differences from original (Low: 0.028423, Medium: 0.026889, High: 0.026414):")
print(medians - pd.Series({'Low': 0.028423, 'Medium': 0.026889, 'High': 0.026414}))

# Export to unique CSV and Parquet
output_csv = r'C:/Users/srsch/OneDrive/Desktop/steel_weather_data_restored_v1.csv'
output_parquet = r'C:/Users/srsch/OneDrive/Desktop/steel_data_restored_v1.parquet'
df_no_outliers.to_csv(output_csv, index=False)
df_no_outliers.to_parquet(output_parquet)
print(f"Exported to {output_csv} and {output_parquet}")

Loaded C:/Users/srsch/OneDrive/Desktop/merged_steel_weather_data_with_datetime.csv, shape: (35040, 20)
Input columns: ['date', 'Usage_kWh', 'Lagging_Current_Reactive.Power_kVarh', 'Leading_Current_Reactive_Power_kVarh', 'CO2(tCO2)', 'Lagging_Current_Power_Factor', 'Leading_Current_Power_Factor', 'NSM', 'WeekStatus', 'Day_of_week', 'Load_Type', 'date_only', 'time_only', 'time_numeric_dec', 'time_seconds_since_midnight', 'temperature_2m (°C)', 'relative_humidity_2m (%)', 'dew_point_2m (°C)', 'precipitation (mm)', 'Load_Type_Rank']
Humidity bins: ['Medium', 'Low', 'High']
Categories (3, object): ['Low' < 'Medium' < 'High']
Shape: (25247, 41)
Columns: ['date', 'Usage_kWh', 'Lagging_Current_Reactive.Power_kVarh', 'Leading_Current_Reactive_Power_kVarh', 'CO2(tCO2)', 'Lagging_Current_Power_Factor', 'Leading_Current_Power_Factor', 'NSM', 'WeekStatus', 'Day_of_week', 'Load_Type', 'date_only', 'time_only', 'time_numeric_dec', 'time_seconds_since_midnight', 'temperature_2m (°C)', 'relative_humidi

C:\Users\srsch\AppData\Local\Temp\ipykernel_26556\2191669122.py:26: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df['CO2_imputed'] = df.groupby('humidity_binned')['CO2(tCO2)'].transform(
C:\Users\srsch\AppData\Local\Temp\ipykernel_26556\2191669122.py:63: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  medians = df_no_outliers.groupby('humidity_binned')['CO2_imputed'].median()


Exported to C:/Users/srsch/OneDrive/Desktop/steel_weather_data_restored_v1.csv and C:/Users/srsch/OneDrive/Desktop/steel_data_restored_v1.parquet


In [5]:
import pandas as pd
import numpy as np

# Load dataset
file_path = r'C:/Users/srsch/OneDrive/Desktop/merged_steel_weather_data_with_datetime.csv'
try:
    df = pd.read_csv(file_path)
    print(f"Loaded {file_path}, shape: {df.shape}")
except FileNotFoundError:
    print(f"{file_path} not found!")
    exit()

# Verify required columns
required_cols = ['CO2(tCO2)', 'Usage_kWh', 'date', 'temperature_2m (°C)', 'relative_humidity_2m (%)', 'Day_of_week']
print("Input columns:", df.columns.tolist())
missing_cols = [col for col in required_cols if col not in df.columns]
if missing_cols:
    print("Missing columns:", missing_cols)
    exit()

# Create humidity_binned
df['humidity_binned'] = pd.qcut(df['relative_humidity_2m (%)'], q=3, labels=['Low', 'Medium', 'High'])
print("Humidity bins:", df['humidity_binned'].unique())

# Create CO2_imputed
df['CO2_imputed'] = df.groupby('humidity_binned', observed=False)['CO2(tCO2)'].transform(
    lambda x: x.replace(0, x[x != 0].mean() if x[x != 0].mean() else x)
)

# Remove outliers
Q1 = df['CO2_imputed'].quantile(0.25)
Q3 = df['CO2_imputed'].quantile(0.75)
IQR = Q3 - Q1
df_no_outliers = df[(df['CO2_imputed'] >= Q1 - 1.5 * IQR) & (df['CO2_imputed'] <= Q3 + 1.5 * IQR)].copy()

# Add time-based and lag features
df_no_outliers['date_time'] = pd.to_datetime(df_no_outliers['date'])
df_no_outliers['hour_of_day'] = df_no_outliers['date_time'].dt.hour
df_no_outliers['is_weekend'] = df_no_outliers['Day_of_week'].isin(['Saturday', 'Sunday']).astype(int)
df_no_outliers['year'] = df_no_outliers['date_time'].dt.year
df_no_outliers['month'] = df_no_outliers['date_time'].dt.month
df_no_outliers['day'] = df_no_outliers['date_time'].dt.day
df_no_outliers['quarter'] = df_no_outliers['date_time'].dt.quarter
df_no_outliers['month_name'] = df_no_outliers['date_time'].dt.month_name()
df_no_outliers['day_name'] = df_no_outliers['date_time'].dt.day_name()
df_no_outliers['week'] = df_no_outliers['date_time'].dt.isocalendar().week
df_no_outliers['year_month'] = df_no_outliers['date_time'].dt.to_period('M')
for lag in range(1, 5):
    df_no_outliers[f'CO2_lag_{lag}'] = df_no_outliers['CO2_imputed'].shift(lag)
    df_no_outliers[f'kWh_lag_{lag}'] = df_no_outliers['Usage_kWh'].shift(lag)

# Add dummy variables for Day_of_week
df_no_outliers = pd.get_dummies(df_no_outliers, columns=['Day_of_week'], prefix='day_of_week')

# Add additional features
df_no_outliers['temp_humid_interaction'] = df_no_outliers['temperature_2m (°C)'] * df_no_outliers['relative_humidity_2m (%)']
df_no_outliers['log_kWh'] = np.log1p(df_no_outliers['Usage_kWh'])
df_no_outliers['log_CO2'] = np.log1p(df_no_outliers['CO2(tCO2)'])
df_no_outliers['log_humidity'] = np.log1p(df_no_outliers['relative_humidity_2m (%)'])
df_no_outliers['yj_CO2'] = np.log1p(df_no_outliers['CO2(tCO2)'])  # Adjust if Yeo-Johnson
df_no_outliers['yj_kWh'] = np.log1p(df_no_outliers['Usage_kWh'])
df_no_outliers['qt_CO2'] = df_no_outliers['CO2(tCO2)'].rank(pct=True)
df_no_outliers['qt_kWh'] = df_no_outliers['Usage_kWh'].rank(pct=True)

# Drop rows with NaN lags
df_no_outliers = df_no_outliers.dropna()

# Verify
print("Shape:", df_no_outliers.shape)  # Expect ~25,241
print("Columns:", df_no_outliers.columns.tolist())
print("Sample CO2_imputed:", df_no_outliers['CO2_imputed'].head().tolist())
print("Humidity bins:", df_no_outliers['humidity_binned'].unique())
print("Median CO2_imputed by bin:")
medians = df_no_outliers.groupby('humidity_binned', observed=False)['CO2_imputed'].median()
print(medians)
print("Median differences from original (Low: 0.028423, Medium: 0.026889, High: 0.026414):")
print(medians - pd.Series({'Low': 0.028423, 'Medium': 0.026889, 'High': 0.026414}))

# Export to unique CSV and Parquet
output_csv = r'C:/Users/srsch/OneDrive/Desktop/steel_weather_data_restored_v2.csv'
output_parquet = r'C:/Users/srsch/OneDrive/Desktop/steel_data_restored_v2.parquet'
df_no_outliers.to_csv(output_csv, index=False)
df_no_outliers.to_parquet(output_parquet)
print(f"Exported to {output_csv} and {output_parquet}")

Loaded C:/Users/srsch/OneDrive/Desktop/merged_steel_weather_data_with_datetime.csv, shape: (35040, 20)
Input columns: ['date', 'Usage_kWh', 'Lagging_Current_Reactive.Power_kVarh', 'Leading_Current_Reactive_Power_kVarh', 'CO2(tCO2)', 'Lagging_Current_Power_Factor', 'Leading_Current_Power_Factor', 'NSM', 'WeekStatus', 'Day_of_week', 'Load_Type', 'date_only', 'time_only', 'time_numeric_dec', 'time_seconds_since_midnight', 'temperature_2m (°C)', 'relative_humidity_2m (%)', 'dew_point_2m (°C)', 'precipitation (mm)', 'Load_Type_Rank']
Humidity bins: ['Medium', 'Low', 'High']
Categories (3, object): ['Low' < 'Medium' < 'High']
Shape: (25247, 55)
Columns: ['date', 'Usage_kWh', 'Lagging_Current_Reactive.Power_kVarh', 'Leading_Current_Reactive_Power_kVarh', 'CO2(tCO2)', 'Lagging_Current_Power_Factor', 'Leading_Current_Power_Factor', 'NSM', 'WeekStatus', 'Load_Type', 'date_only', 'time_only', 'time_numeric_dec', 'time_seconds_since_midnight', 'temperature_2m (°C)', 'relative_humidity_2m (%)', 'de

In [6]:
from xgboost import XGBRegressor
from sklearn.metrics import r2_score, mean_squared_error
import pandas as pd

# Load restored dataset
df = pd.read_csv(r'C:/Users/srsch/OneDrive/Desktop/steel_weather_data_restored_v2.csv')

# Define features (based on your original importance)
features = [
    'Usage_kWh', 'temperature_2m (°C)', 'relative_humidity_2m (%)', 'hour_of_day', 
    'is_weekend', 'CO2_lag_1', 'kWh_lag_1', 'CO2_lag_3', 'kWh_lag_3'
]
if 'day_of_week_Monday' in df.columns:
    features.append('day_of_week_Monday')  # Adjust for dummies

# Full model
X = df[features]
y = df['CO2_imputed']
model = XGBRegressor(random_state=42)
model.fit(X, y)
y_pred = model.predict(X)
print("Full R²:", r2_score(y, y_pred))
print("Full MSE:", mean_squared_error(y, y_pred))

# Low Humidity model
df_low = df[df['humidity_binned'] == 'Low']
X_low = df_low[features]
y_low = df_low['CO2_imputed']
model_low = XGBRegressor(random_state=42)
model_low.fit(X_low, y_low)
y_low_pred = model_low.predict(X_low)
print("Low Humidity R²:", r2_score(y_low, y_low_pred))
print("Low Humidity MSE:", mean_squared_error(y_low, y_low_pred))

# Feature Importance
importances = pd.DataFrame({'Feature': features, 'Importance': model_low.feature_importances_})
print(importances.sort_values(by='Importance', ascending=False))
importances.to_csv(r'C:/Users/srsch/OneDrive/Desktop/feature_importance_v2.csv', index=False)

Full R²: 0.9994343208741002
Full MSE: 9.040702926559307e-10
Low Humidity R²: 0.7413370857206096
Low Humidity MSE: 1.312105729020918e-10
                    Feature  Importance
0                 Usage_kWh         1.0
1       temperature_2m (°C)         0.0
2  relative_humidity_2m (%)         0.0
3               hour_of_day         0.0
4                is_weekend         0.0
5                 CO2_lag_1         0.0
6                 kWh_lag_1         0.0
7                 CO2_lag_3         0.0
8                 kWh_lag_3         0.0
9        day_of_week_Monday         0.0


In [7]:
from xgboost import XGBRegressor
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np

# Load dataset
df = pd.read_csv(r'C:/Users/srsch/OneDrive/Desktop/steel_weather_data_restored_v2.csv')
print("Dataset shape:", df.shape)
print("Low humidity rows:", len(df[df['humidity_binned'] == 'Low']))

# Define features (expanded to match original)
features = [
    'Usage_kWh', 'temperature_2m (°C)', 'relative_humidity_2m (%)', 'hour_of_day', 
    'is_weekend', 'CO2_lag_1', 'kWh_lag_1', 'CO2_lag_2', 'kWh_lag_2', 
    'CO2_lag_3', 'kWh_lag_3', 'CO2_lag_4', 'kWh_lag_4', 
    'temp_humid_interaction', 'log_kWh', 'log_humidity'
]
day_of_week_cols = [col for col in df.columns if col.startswith('day_of_week_')]
features.extend(day_of_week_cols)
print("Features used:", features)

# Check correlations
corr_matrix = df[features].corr()
high_corr = corr_matrix[abs(corr_matrix) > 0.8].stack().reset_index()
high_corr = high_corr[high_corr['level_0'] != high_corr['level_1']]
print("High correlations (|r| > 0.8):")
print(high_corr)

# Low Humidity subset
df_low = df[df['humidity_binned'] == 'Low'].copy()
X_low = df_low[features]
y_low = df_low['CO2_imputed']

# Scale features
scaler = StandardScaler()
X_low_scaled = scaler.fit_transform(X_low)
X_low_scaled = pd.DataFrame(X_low_scaled, columns=features, index=X_low.index)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_low_scaled, y_low, test_size=0.2, random_state=42)

# Train model with tuned parameters
model_low = XGBRegressor(
    n_estimators=100, max_depth=3, learning_rate=0.1, 
    min_child_weight=1, subsample=0.8, colsample_bytree=0.8, random_state=42
)
model_low.fit(X_train, y_train)

# Evaluate
y_train_pred = model_low.predict(X_train)
y_test_pred = model_low.predict(X_test)
print("Low Humidity Train R²:", r2_score(y_train, y_train_pred))
print("Low Humidity Test R²:", r2_score(y_test, y_test_pred))
print("Low Humidity Train MSE:", mean_squared_error(y_train, y_train_pred))
print("Low Humidity Test MSE:", mean_squared_error(y_test, y_test_pred))

# Feature Importance
importances = pd.DataFrame({'Feature': features, 'Importance': model_low.feature_importances_})
print("Feature Importance:")
print(importances.sort_values('Importance', ascending=False))
importances.to_csv(r'C:/Users/srsch/OneDrive/Desktop/feature_importance_low_xgboost_v4.csv', index=False)

Dataset shape: (25247, 55)
Low humidity rows: 7577
Features used: ['Usage_kWh', 'temperature_2m (°C)', 'relative_humidity_2m (%)', 'hour_of_day', 'is_weekend', 'CO2_lag_1', 'kWh_lag_1', 'CO2_lag_2', 'kWh_lag_2', 'CO2_lag_3', 'kWh_lag_3', 'CO2_lag_4', 'kWh_lag_4', 'temp_humid_interaction', 'log_kWh', 'log_humidity', 'day_of_week_Friday', 'day_of_week_Monday', 'day_of_week_Saturday', 'day_of_week_Sunday', 'day_of_week_Thursday', 'day_of_week_Tuesday', 'day_of_week_Wednesday']
High correlations (|r| > 0.8):
                     level_0                   level_1         0
1                  Usage_kWh                 kWh_lag_1  0.860765
2                  Usage_kWh                   log_kWh  0.986016
4        temperature_2m (°C)    temp_humid_interaction  0.936631
6   relative_humidity_2m (%)              log_humidity  0.972684
10                 CO2_lag_1                 CO2_lag_2  0.929752
11                 CO2_lag_1                 CO2_lag_3  0.869118
12                 CO2_lag_1       

In [8]:
['Usage_kWh', 'temperature_2m (°C)', 'relative_humidity_2m (%)', 'hour_of_day', 'is_weekend', 'CO2_lag_1', 'kWh_lag_1', 'CO2_lag_2', 'kWh_lag_2', 'CO2_lag_3', 'kWh_lag_3', 'CO2_lag_4', 'kWh_lag_4', 'temp_humid_interaction', 'log_kWh', 'log_humidity', 'day_of_week_Friday', 'day_of_week_Monday', 'day_of_week_Saturday', 'day_of_week_Sunday', 'day_of_week_Thursday', 'day_of_week_Tuesday', 'day_of_week_Wednesday']

['Usage_kWh',
 'temperature_2m (°C)',
 'relative_humidity_2m (%)',
 'hour_of_day',
 'is_weekend',
 'CO2_lag_1',
 'kWh_lag_1',
 'CO2_lag_2',
 'kWh_lag_2',
 'CO2_lag_3',
 'kWh_lag_3',
 'CO2_lag_4',
 'kWh_lag_4',
 'temp_humid_interaction',
 'log_kWh',
 'log_humidity',
 'day_of_week_Friday',
 'day_of_week_Monday',
 'day_of_week_Saturday',
 'day_of_week_Sunday',
 'day_of_week_Thursday',
 'day_of_week_Tuesday',
 'day_of_week_Wednesday']

In [9]:
['Usage_kWh', 'temperature_2m (°C)', 'relative_humidity_2m (%)', 'hour_of_day', 'is_weekend', 'CO2_lag_1', 'kWh_lag_1', 'CO2_lag_2', 'kWh_lag_2', 'CO2_lag_3', 'kWh_lag_3', 'CO2_lag_4', 'kWh_lag_4', 'temp_humid_interaction', 'log_kWh', 'log_humidity', 'day_of_week_Friday', 'day_of_week_Monday', 'day_of_week_Saturday', 'day_of_week_Sunday', 'day_of_week_Thursday', 'day_of_week_Tuesday', 'day_of_week_Wednesday']

['Usage_kWh',
 'temperature_2m (°C)',
 'relative_humidity_2m (%)',
 'hour_of_day',
 'is_weekend',
 'CO2_lag_1',
 'kWh_lag_1',
 'CO2_lag_2',
 'kWh_lag_2',
 'CO2_lag_3',
 'kWh_lag_3',
 'CO2_lag_4',
 'kWh_lag_4',
 'temp_humid_interaction',
 'log_kWh',
 'log_humidity',
 'day_of_week_Friday',
 'day_of_week_Monday',
 'day_of_week_Saturday',
 'day_of_week_Sunday',
 'day_of_week_Thursday',
 'day_of_week_Tuesday',
 'day_of_week_Wednesday']

In [10]:
from xgboost import XGBRegressor
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectFromModel
import pandas as pd
import numpy as np

# Load dataset
df = pd.read_csv(r'C:/Users/srsch/OneDrive/Desktop/steel_weather_data_restored_v2.csv')
print("Dataset shape:", df.shape)
print("Low humidity rows:", len(df[df['humidity_binned'] == 'Low']))

# Define features (remove redundant: log_kWh, log_humidity, higher lags)
features = [
    'Usage_kWh', 'temperature_2m (°C)', 'relative_humidity_2m (%)', 'hour_of_day', 
    'is_weekend', 'CO2_lag_1', 'kWh_lag_1', 'temp_humid_interaction'
]
day_of_week_cols = [col for col in df.columns if col.startswith('day_of_week_')]
features.extend(day_of_week_cols)
print("Features used:", features)

# Low Humidity subset
df_low = df[df['humidity_binned'] == 'Low'].copy()
X_low = df_low[features]
y_low = df_low['CO2_imputed']

# Check feature statistics
print("Feature means:", X_low.mean())
print("Feature variances:", X_low.var())

# Scale features
scaler = StandardScaler()
X_low_scaled = scaler.fit_transform(X_low)
X_low_scaled = pd.DataFrame(X_low_scaled, columns=features, index=X_low.index)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_low_scaled, y_low, test_size=0.2, random_state=42)

# Feature selection
initial_model = XGBRegressor(random_state=42)
initial_model.fit(X_train, y_train)
selector = SelectFromModel(initial_model, threshold='0.5*mean', prefit=True)
X_train_selected = selector.transform(X_train)
X_test_selected = selector.transform(X_test)
selected_features = X_low_scaled.columns[selector.get_support()].tolist()
print("Selected features:", selected_features)

# Train final model
model_low = XGBRegressor(
    n_estimators=200, max_depth=5, learning_rate=0.05, 
    min_child_weight=2, subsample=0.9, colsample_bytree=0.9, random_state=42
)
model_low.fit(X_train_selected, y_train)

# Evaluate
y_train_pred = model_low.predict(X_train_selected)
y_test_pred = model_low.predict(X_test_selected)
print("Low Humidity Train R²:", r2_score(y_train, y_train_pred))
print("Low Humidity Test R²:", r2_score(y_test, y_test_pred))
print("Low Humidity Train MSE:", mean_squared_error(y_train, y_train_pred))
print("Low Humidity Test MSE:", mean_squared_error(y_test, y_test_pred))

# Feature Importance
importances = pd.DataFrame({'Feature': selected_features, 'Importance': model_low.feature_importances_})
print("Feature Importance:")
print(importances.sort_values('Importance', ascending=False))
importances.to_csv(r'C:/Users/srsch/OneDrive/Desktop/feature_importance_low_xgboost_v5.csv', index=False)

Dataset shape: (25247, 55)
Low humidity rows: 7577
Features used: ['Usage_kWh', 'temperature_2m (°C)', 'relative_humidity_2m (%)', 'hour_of_day', 'is_weekend', 'CO2_lag_1', 'kWh_lag_1', 'temp_humid_interaction', 'day_of_week_Friday', 'day_of_week_Monday', 'day_of_week_Saturday', 'day_of_week_Sunday', 'day_of_week_Thursday', 'day_of_week_Tuesday', 'day_of_week_Wednesday']
Feature means: Usage_kWh                    20.927528
temperature_2m (°C)           9.338894
relative_humidity_2m (%)     48.056289
hour_of_day                  12.016233
is_weekend                    0.356078
CO2_lag_1                     0.029909
kWh_lag_1                    20.757622
temp_humid_interaction      483.442939
day_of_week_Friday            0.154415
day_of_week_Monday            0.135410
day_of_week_Saturday          0.163653
day_of_week_Sunday            0.192424
day_of_week_Thursday          0.136334
day_of_week_Tuesday           0.104659
day_of_week_Wednesday         0.113105
dtype: float64
Feature var

C:\Users\srsch\anaconda3\Lib\site-packages\sklearn\utils\validation.py:2732: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
C:\Users\srsch\anaconda3\Lib\site-packages\sklearn\utils\validation.py:2732: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(


In [11]:

#Fix Low Humidity XGBoost Model
#Let’s improve the model by:Removing feature selection (since it’s dropping critical features).
#Keeping minimal features with variance (e.g., Usage_kWh, CO2_lag_1, day_of_week_).
#Adjusting hyperparameters for better fit. Checking CO2_imputed variance in the Low humidity subset.
#fixing the SelectFromModel warning.

from xgboost import XGBRegressor
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np

# Load dataset
df = pd.read_csv(r'C:/Users/srsch/OneDrive/Desktop/steel_weather_data_restored_v2.csv')
print("Dataset shape:", df.shape)
print("Low humidity rows:", len(df[df['humidity_binned'] == 'Low']))
print("CO2_imputed variance (Low humidity):", df[df['humidity_binned'] == 'Low']['CO2_imputed'].var())

# Define features (minimal, high-variance, non-redundant)
features = [
    'Usage_kWh', 'temperature_2m (°C)', 'relative_humidity_2m (%)', 'hour_of_day', 
    'is_weekend', 'CO2_lag_1'
]
day_of_week_cols = [col for col in df.columns if col.startswith('day_of_week_')]
features.extend(day_of_week_cols)
print("Features used:", features)

# Low Humidity subset
df_low = df[df['humidity_binned'] == 'Low'].copy()
X_low = df_low[features]
y_low = df_low['CO2_imputed']

# Check feature statistics
print("Feature means:", X_low.mean())
print("Feature variances:", X_low.var())

# Scale features
scaler = StandardScaler()
X_low_scaled = scaler.fit_transform(X_low)
X_low_scaled = pd.DataFrame(X_low_scaled, columns=features, index=X_low.index)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_low_scaled, y_low, test_size=0.2, random_state=42)

# Train model (no feature selection)
model_low = XGBRegressor(
    n_estimators=300, max_depth=6, learning_rate=0.03, 
    min_child_weight=1, subsample=0.8, colsample_bytree=0.8, random_state=42
)
model_low.fit(X_train, y_train)

# Evaluate
y_train_pred = model_low.predict(X_train)
y_test_pred = model_low.predict(X_test)
print("Low Humidity Train R²:", r2_score(y_train, y_train_pred))
print("Low Humidity Test R²:", r2_score(y_test, y_test_pred))
print("Low Humidity Train MSE:", mean_squared_error(y_train, y_train_pred))
print("Low Humidity Test MSE:", mean_squared_error(y_test, y_test_pred))

# Feature Importance
importances = pd.DataFrame({'Feature': features, 'Importance': model_low.feature_importances_})
print("Feature Importance:")
print(importances.sort_values('Importance', ascending=False))
importances.to_csv(r'C:/Users/srsch/OneDrive/Desktop/feature_importance_low_xgboost_v6.csv', index=False)

Dataset shape: (25247, 55)
Low humidity rows: 7577
CO2_imputed variance (Low humidity): 5.073316849811862e-10
Features used: ['Usage_kWh', 'temperature_2m (°C)', 'relative_humidity_2m (%)', 'hour_of_day', 'is_weekend', 'CO2_lag_1', 'day_of_week_Friday', 'day_of_week_Monday', 'day_of_week_Saturday', 'day_of_week_Sunday', 'day_of_week_Thursday', 'day_of_week_Tuesday', 'day_of_week_Wednesday']
Feature means: Usage_kWh                   20.927528
temperature_2m (°C)          9.338894
relative_humidity_2m (%)    48.056289
hour_of_day                 12.016233
is_weekend                   0.356078
CO2_lag_1                    0.029909
day_of_week_Friday           0.154415
day_of_week_Monday           0.135410
day_of_week_Saturday         0.163653
day_of_week_Sunday           0.192424
day_of_week_Thursday         0.136334
day_of_week_Tuesday          0.104659
day_of_week_Wednesday        0.113105
dtype: float64
Feature variances: Usage_kWh                   7.516623e+02
temperature_2m (°C)   

In [12]:
import pandas as pd

# Load dataset
df = pd.read_csv(r'C:/Users/srsch/OneDrive/Desktop/steel_weather_data_restored_v2.csv')
print("Dataset shape:", df.shape)
print("Columns:", df.columns.tolist())

# Inspect CO2_imputed
print("\nCO2_imputed summary:")
print(df['CO2_imputed'].describe())
print("CO2_imputed variance by humidity bin:")
print(df.groupby('humidity_binned')['CO2_imputed'].var())

# Inspect humidity_binned
print("\nhumidity_binned value counts:")
print(df['humidity_binned'].value_counts())
print("relative_humidity_2m (%) range by bin:")
print(df.groupby('humidity_binned')['relative_humidity_2m (%)'].agg(['min', 'max']))

# Compare with original medians
original_medians = {'Low': 0.028423, 'Medium': 0.026889, 'High': 0.026414}
current_medians = df.groupby('humidity_binned')['CO2_imputed'].median()
print("\nCurrent vs. Original CO2_imputed medians:")
print(pd.DataFrame({'Current': current_medians, 'Original': pd.Series(original_medians)}))

# Check original dataset if available
try:
    df_orig = pd.read_csv(r'C:/Users/srsch/steel_weather_data.csv')  # Truncated file (15,171 rows)
    print("\nOriginal dataset shape:", df_orig.shape)
    print("Original CO2_imputed variance (Low humidity):", df_orig[df_orig['humidity_binned'] == 'Low']['CO2_imputed'].var())
    print("Original humidity_binned value counts:")
    print(df_orig['humidity_binned'].value_counts())
except FileNotFoundError:
    print("\nOriginal dataset not found at C:/Users/srsch/steel_weather_data.csv")

Dataset shape: (25247, 55)
Columns: ['date', 'Usage_kWh', 'Lagging_Current_Reactive.Power_kVarh', 'Leading_Current_Reactive_Power_kVarh', 'CO2(tCO2)', 'Lagging_Current_Power_Factor', 'Leading_Current_Power_Factor', 'NSM', 'WeekStatus', 'Load_Type', 'date_only', 'time_only', 'time_numeric_dec', 'time_seconds_since_midnight', 'temperature_2m (°C)', 'relative_humidity_2m (%)', 'dew_point_2m (°C)', 'precipitation (mm)', 'Load_Type_Rank', 'humidity_binned', 'CO2_imputed', 'date_time', 'hour_of_day', 'is_weekend', 'year', 'month', 'day', 'quarter', 'month_name', 'day_name', 'week', 'year_month', 'CO2_lag_1', 'kWh_lag_1', 'CO2_lag_2', 'kWh_lag_2', 'CO2_lag_3', 'kWh_lag_3', 'CO2_lag_4', 'kWh_lag_4', 'day_of_week_Friday', 'day_of_week_Monday', 'day_of_week_Saturday', 'day_of_week_Sunday', 'day_of_week_Thursday', 'day_of_week_Tuesday', 'day_of_week_Wednesday', 'temp_humid_interaction', 'log_kWh', 'log_CO2', 'log_humidity', 'yj_CO2', 'yj_kWh', 'qt_CO2', 'qt_kWh']

CO2_imputed summary:
count    25

In [13]:
import pandas as pd

# Load source dataset
source_file = r'C:/Users/srsch/OneDrive/Desktop/merged_steel_weather_data_with_datetime.csv'
try:
    df_source = pd.read_csv(source_file)
    print("Source dataset shape:", df_source.shape)
    print("Columns:", df_source.columns.tolist())

    # Inspect CO2(tCO2)
    print("\nCO2(tCO2) summary:")
    print(df_source['CO2(tCO2)'].describe())
    print("CO2(tCO2) zero count:", (df_source['CO2(tCO2)'] == 0).sum())
    print("CO2(tCO2) non-zero count:", (df_source['CO2(tCO2)'] != 0).sum())
    print("CO2(tCO2) non-zero values (sample):")
    print(df_source[df_source['CO2(tCO2)'] != 0]['CO2(tCO2)'].head(10))
    print("CO2(tCO2) variance:", df_source['CO2(tCO2)'].var())

    # Check for missing values
    print("\nMissing values:")
    print(df_source[['CO2(tCO2)', 'Usage_kWh', 'relative_humidity_2m (%)']].isnull().sum())
except FileNotFoundError:
    print(f"{source_file} not found!")

Source dataset shape: (35040, 20)
Columns: ['date', 'Usage_kWh', 'Lagging_Current_Reactive.Power_kVarh', 'Leading_Current_Reactive_Power_kVarh', 'CO2(tCO2)', 'Lagging_Current_Power_Factor', 'Leading_Current_Power_Factor', 'NSM', 'WeekStatus', 'Day_of_week', 'Load_Type', 'date_only', 'time_only', 'time_numeric_dec', 'time_seconds_since_midnight', 'temperature_2m (°C)', 'relative_humidity_2m (%)', 'dew_point_2m (°C)', 'precipitation (mm)', 'Load_Type_Rank']

CO2(tCO2) summary:
count    35040.000000
mean         0.011524
std          0.016151
min          0.000000
25%          0.000000
50%          0.000000
75%          0.020000
max          0.070000
Name: CO2(tCO2), dtype: float64
CO2(tCO2) zero count: 20990
CO2(tCO2) non-zero count: 14050
CO2(tCO2) non-zero values (sample):
224    0.02
225    0.03
226    0.05
227    0.05
228    0.06
229    0.05
230    0.03
231    0.02
232    0.03
233    0.03
Name: CO2(tCO2), dtype: float64
CO2(tCO2) variance: 0.00026084903623702327

Missing values:
CO2(

In [14]:
import pandas as pd
import numpy as np

# Load source dataset
source_file = r'C:/Users/srsch/OneDrive/Desktop/merged_steel_weather_data_with_datetime.csv'
df = pd.read_csv(source_file)
print("Source shape:", df.shape)

# Create humidity_binned
df['humidity_binned'] = pd.qcut(df['relative_humidity_2m (%)'], q=3, labels=['Low', 'Medium', 'High'])

# Impute CO2(tCO2) zeros with bin-specific medians
df['CO2_imputed'] = df.groupby('humidity_binned')['CO2(tCO2)'].transform(
    lambda x: x.replace(0, x[x != 0].median() if x[x != 0].size > 0 else np.nan)
)

# Remove outliers (IQR method for CO2_imputed)
Q1 = df['CO2_imputed'].quantile(0.25)
Q3 = df['CO2_imputed'].quantile(0.75)
IQR = Q3 - Q1
df = df[(df['CO2_imputed'] >= Q1 - 1.5 * IQR) & (df['CO2_imputed'] <= Q3 + 1.5 * IQR)]

# Add time-based features
df['date_time'] = pd.to_datetime(df['date'])
df['hour_of_day'] = df['date_time'].dt.hour
df['is_weekend'] = df['Day_of_week'].isin(['Saturday', 'Sunday']).astype(int)
df['year'] = df['date_time'].dt.year
df['month'] = df['date_time'].dt.month
df['day'] = df['date_time'].dt.day
df['month_name'] = df['date_time'].dt.strftime('%B')
df['day_name'] = df['Day_of_week']
df['week'] = df['date_time'].dt.isocalendar().week
df['year_month'] = df['date_time'].dt.strftime('%Y-%m')

# Add lags
for lag in range(1, 5):
    df[f'CO2_lag_{lag}'] = df['CO2_imputed'].shift(lag)
    df[f'kWh_lag_{lag}'] = df['Usage_kWh'].shift(lag)

# Add interaction and transformations
df['temp_humid_interaction'] = df['temperature_2m (°C)'] * df['relative_humidity_2m (%)']
df['log_kWh'] = np.log1p(df['Usage_kWh'])
df['log_humidity'] = np.log1p(df['relative_humidity_2m (%)'])

# Add day_of_week dummies
df = pd.get_dummies(df, columns=['Day_of_week'], prefix='day_of_week')

# Drop rows with NaN (from lags or imputation)
df = df.dropna()

# Verify
print("New shape:", df.shape)
print("Columns:", df.columns.tolist())
print("CO2_imputed summary:")
print(df['CO2_imputed'].describe())
print("CO2_imputed variance by humidity bin:")
print(df.groupby('humidity_binned')['CO2_imputed'].var())
print("humidity_binned value counts:")
print(df['humidity_binned'].value_counts())
print("relative_humidity_2m (%) range by bin:")
print(df.groupby('humidity_binned')['relative_humidity_2m (%)'].agg(['min', 'max']))
print("CO2_imputed medians:")
print(df.groupby('humidity_binned')['CO2_imputed'].median())

# Save updated dataset
output_csv = r'C:/Users/srsch/OneDrive/Desktop/steel_weather_data_restored_v4.csv'
df.to_csv(output_csv, index=False)
print(f"Updated dataset saved to {output_csv}")


Source shape: (35040, 20)


C:\Users\srsch\AppData\Local\Temp\ipykernel_26556\1463559869.py:13: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df['CO2_imputed'] = df.groupby('humidity_binned')['CO2(tCO2)'].transform(


New shape: (33236, 49)
Columns: ['date', 'Usage_kWh', 'Lagging_Current_Reactive.Power_kVarh', 'Leading_Current_Reactive_Power_kVarh', 'CO2(tCO2)', 'Lagging_Current_Power_Factor', 'Leading_Current_Power_Factor', 'NSM', 'WeekStatus', 'Load_Type', 'date_only', 'time_only', 'time_numeric_dec', 'time_seconds_since_midnight', 'temperature_2m (°C)', 'relative_humidity_2m (%)', 'dew_point_2m (°C)', 'precipitation (mm)', 'Load_Type_Rank', 'humidity_binned', 'CO2_imputed', 'date_time', 'hour_of_day', 'is_weekend', 'year', 'month', 'day', 'month_name', 'day_name', 'week', 'year_month', 'CO2_lag_1', 'kWh_lag_1', 'CO2_lag_2', 'kWh_lag_2', 'CO2_lag_3', 'kWh_lag_3', 'CO2_lag_4', 'kWh_lag_4', 'temp_humid_interaction', 'log_kWh', 'log_humidity', 'day_of_week_Friday', 'day_of_week_Monday', 'day_of_week_Saturday', 'day_of_week_Sunday', 'day_of_week_Thursday', 'day_of_week_Tuesday', 'day_of_week_Wednesday']
CO2_imputed summary:
count    33236.000000
mean         0.025799
std          0.006671
min         

C:\Users\srsch\AppData\Local\Temp\ipykernel_26556\1463559869.py:57: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  print(df.groupby('humidity_binned')['CO2_imputed'].var())
C:\Users\srsch\AppData\Local\Temp\ipykernel_26556\1463559869.py:61: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  print(df.groupby('humidity_binned')['relative_humidity_2m (%)'].agg(['min', 'max']))
C:\Users\srsch\AppData\Local\Temp\ipykernel_26556\1463559869.py:63: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=

Updated dataset saved to C:/Users/srsch/OneDrive/Desktop/steel_weather_data_restored_v4.csv


In [15]:
from xgboost import XGBRegressor
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd

# Load updated dataset
df = pd.read_csv(r'C:/Users/srsch/OneDrive/Desktop/steel_weather_data_restored_v4.csv')
print("Dataset shape:", df.shape)
print("Low humidity rows:", len(df[df['humidity_binned'] == 'Low']))

# Define features
features = [
    'Usage_kWh', 'temperature_2m (°C)', 'relative_humidity_2m (%)', 'hour_of_day', 
    'is_weekend', 'CO2_lag_1', 'temp_humid_interaction', 'log_kWh'
]
day_of_week_cols = [col for col in df.columns if col.startswith('day_of_week_')]
features.extend(day_of_week_cols)
print("Features used:", features)

# Low Humidity subset
df_low = df[df['humidity_binned'] == 'Low'].copy()
X_low = df_low[features]
y_low = df_low['CO2_imputed']

# Check feature statistics
print("Feature variances:", X_low.var())

# Scale features
scaler = StandardScaler()
X_low_scaled = scaler.fit_transform(X_low)
X_low_scaled = pd.DataFrame(X_low_scaled, columns=features, index=X_low.index)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_low_scaled, y_low, test_size=0.2, random_state=42)

# Train model
model_low = XGBRegressor(
    n_estimators=500, max_depth=6, learning_rate=0.02, 
    min_child_weight=1, subsample=0.8, colsample_bytree=0.8, random_state=42
)
model_low.fit(X_train, y_train)

# Evaluate
y_train_pred = model_low.predict(X_train)
y_test_pred = model_low.predict(X_test)
print("Low Humidity Train R²:", r2_score(y_train, y_train_pred))
print("Low Humidity Test R²:", r2_score(y_test, y_test_pred))
print("Low Humidity Train MSE:", mean_squared_error(y_train, y_train_pred))
print("Low Humidity Test MSE:", mean_squared_error(y_test, y_test_pred))

# Feature Importance
importances = pd.DataFrame({'Feature': features, 'Importance': model_low.feature_importances_})
print("Feature Importance:")
print(importances.sort_values('Importance', ascending=False))
importances.to_csv(r'C:/Users/srsch/OneDrive/Desktop/feature_importance_low_xgboost_v9.csv', index=False)

Dataset shape: (33236, 49)
Low humidity rows: 11050
Features used: ['Usage_kWh', 'temperature_2m (°C)', 'relative_humidity_2m (%)', 'hour_of_day', 'is_weekend', 'CO2_lag_1', 'temp_humid_interaction', 'log_kWh', 'day_of_week_Friday', 'day_of_week_Monday', 'day_of_week_Saturday', 'day_of_week_Sunday', 'day_of_week_Thursday', 'day_of_week_Tuesday', 'day_of_week_Wednesday']
Feature variances: Usage_kWh                      883.239016
temperature_2m (°C)            119.629685
relative_humidity_2m (%)       120.411487
hour_of_day                     32.653978
is_weekend                       0.216767
CO2_lag_1                        0.000044
temp_humid_interaction      352088.112690
log_kWh                          1.501451
day_of_week_Friday               0.138257
day_of_week_Monday               0.114879
day_of_week_Saturday             0.136617
day_of_week_Sunday               0.130565
day_of_week_Thursday             0.126591
day_of_week_Tuesday              0.100405
day_of_week_Wednesda

In [16]:
import pandas as pd
import numpy as np

# Load dataset
df = pd.read_csv(r'C:/Users/srsch/OneDrive/Desktop/steel_weather_data_restored_v4.csv')
print("Dataset shape:", df.shape)
print("Columns:", df.columns.tolist())

# Inspect CO2_imputed
print("\nCO2_imputed summary:")
print(df['CO2_imputed'].describe())
print("CO2_imputed variance by humidity bin:")
print(df.groupby('humidity_binned')['CO2_imputed'].var())
print("humidity_binned value counts:")
print(df['humidity_binned'].value_counts())
print("relative_humidity_2m (%) range by bin:")
print(df.groupby('humidity_binned')['relative_humidity_2m (%)'].agg(['min', 'max']))
print("CO2_imputed medians:")
current_medians = df.groupby('humidity_binned')['CO2_imputed'].median()
original_medians = {'Low': 0.028423, 'Medium': 0.026889, 'High': 0.026414}
print(pd.DataFrame({'Current': current_medians, 'Original': pd.Series(original_medians)}))

# Stricter outlier removal (target ~25,241 rows)
Q1 = df['CO2_imputed'].quantile(0.25)
Q3 = df['CO2_imputed'].quantile(0.75)
IQR = Q3 - Q1
df_clean = df[(df['CO2_imputed'] >= Q1 - 1.25 * IQR) & (df['CO2_imputed'] <= Q3 + 1.25 * IQR)]
print("\nShape after stricter outlier removal:", df_clean.shape)

# Save adjusted dataset
output_csv = r'C:/Users/srsch/OneDrive/Desktop/steel_weather_data_restored_v5.csv'
df_clean.to_csv(output_csv, index=False)
print(f"Adjusted dataset saved to {output_csv}")

Dataset shape: (33236, 49)
Columns: ['date', 'Usage_kWh', 'Lagging_Current_Reactive.Power_kVarh', 'Leading_Current_Reactive_Power_kVarh', 'CO2(tCO2)', 'Lagging_Current_Power_Factor', 'Leading_Current_Power_Factor', 'NSM', 'WeekStatus', 'Load_Type', 'date_only', 'time_only', 'time_numeric_dec', 'time_seconds_since_midnight', 'temperature_2m (°C)', 'relative_humidity_2m (%)', 'dew_point_2m (°C)', 'precipitation (mm)', 'Load_Type_Rank', 'humidity_binned', 'CO2_imputed', 'date_time', 'hour_of_day', 'is_weekend', 'year', 'month', 'day', 'month_name', 'day_name', 'week', 'year_month', 'CO2_lag_1', 'kWh_lag_1', 'CO2_lag_2', 'kWh_lag_2', 'CO2_lag_3', 'kWh_lag_3', 'CO2_lag_4', 'kWh_lag_4', 'temp_humid_interaction', 'log_kWh', 'log_humidity', 'day_of_week_Friday', 'day_of_week_Monday', 'day_of_week_Saturday', 'day_of_week_Sunday', 'day_of_week_Thursday', 'day_of_week_Tuesday', 'day_of_week_Wednesday']

CO2_imputed summary:
count    33236.000000
mean         0.025799
std          0.006671
min    

In [17]:
import pandas as pd
import numpy as np

# Load source dataset
source_file = r'C:/Users/srsch/OneDrive/Desktop/merged_steel_weather_data_with_datetime.csv'
df = pd.read_csv(source_file)
print("Source shape:", df.shape)

# Create humidity_binned
df['humidity_binned'] = pd.qcut(df['relative_humidity_2m (%)'], q=3, labels=['Low', 'Medium', 'High'])

# Impute CO2(tCO2) zeros with bin-specific medians
df['CO2_imputed'] = df.groupby('humidity_binned')['CO2(tCO2)'].transform(
    lambda x: x.replace(0, x[x != 0].median() if x[x != 0].size > 0 else np.nan)
)

# Remove outliers (percentile-based for CO2_imputed and Usage_kWh)
co2_lower, co2_upper = df['CO2_imputed'].quantile([0.05, 0.95])
kwh_lower, kwh_upper = df['Usage_kWh'].quantile([0.05, 0.95])
df = df[
    (df['CO2_imputed'].between(co2_lower, co2_upper)) &
    (df['Usage_kWh'].between(kwh_lower, kwh_upper))
]

# Add time-based features
df['date_time'] = pd.to_datetime(df['date'])
df['hour_of_day'] = df['date_time'].dt.hour
df['is_weekend'] = df['Day_of_week'].isin(['Saturday', 'Sunday']).astype(int)
df['year'] = df['date_time'].dt.year
df['month'] = df['date_time'].dt.month
df['day'] = df['date_time'].dt.day
df['month_name'] = df['date_time'].dt.strftime('%B')
df['day_name'] = df['Day_of_week']
df['week'] = df['date_time'].dt.isocalendar().week
df['year_month'] = df['date_time'].dt.strftime('%Y-%m')

# Add lags
for lag in range(1, 5):
    df[f'CO2_lag_{lag}'] = df['CO2_imputed'].shift(lag)
    df[f'kWh_lag_{lag}'] = df['Usage_kWh'].shift(lag)

# Add interaction and transformations
df['temp_humid_interaction'] = df['temperature_2m (°C)'] * df['relative_humidity_2m (%)']
df['log_kWh'] = np.log1p(df['Usage_kWh'])
df['log_humidity'] = np.log1p(df['relative_humidity_2m (%)'])

# Add day_of_week dummies
df = pd.get_dummies(df, columns=['Day_of_week'], prefix='day_of_week')

# Drop rows with NaN
df = df.dropna()

# Verify
print("New shape:", df.shape)
print("Columns:", df.columns.tolist())
print("CO2_imputed summary:")
print(df['CO2_imputed'].describe())
print("CO2_imputed variance by humidity bin:")
print(df.groupby('humidity_binned')['CO2_imputed'].var())
print("humidity_binned value counts:")
print(df['humidity_binned'].value_counts())
print("relative_humidity_2m (%) range by bin:")
print(df.groupby('humidity_binned')['relative_humidity_2m (%)'].agg(['min', 'max']))
print("CO2_imputed medians:")
current_medians = df.groupby('humidity_binned')['CO2_imputed'].median()
original_medians = {'Low': 0.028423, 'Medium': 0.026889, 'High': 0.026414}
print(pd.DataFrame({'Current': current_medians, 'Original': pd.Series(original_medians)}))

# Save updated dataset
output_csv = r'C:/Users/srsch/OneDrive/Desktop/steel_weather_data_restored_v6.csv'
df.to_csv(output_csv, index=False)
print(f"Updated dataset saved to {output_csv}")

Source shape: (35040, 20)


C:\Users\srsch\AppData\Local\Temp\ipykernel_26556\789275472.py:13: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df['CO2_imputed'] = df.groupby('humidity_binned')['CO2(tCO2)'].transform(


New shape: (30365, 49)
Columns: ['date', 'Usage_kWh', 'Lagging_Current_Reactive.Power_kVarh', 'Leading_Current_Reactive_Power_kVarh', 'CO2(tCO2)', 'Lagging_Current_Power_Factor', 'Leading_Current_Power_Factor', 'NSM', 'WeekStatus', 'Load_Type', 'date_only', 'time_only', 'time_numeric_dec', 'time_seconds_since_midnight', 'temperature_2m (°C)', 'relative_humidity_2m (%)', 'dew_point_2m (°C)', 'precipitation (mm)', 'Load_Type_Rank', 'humidity_binned', 'CO2_imputed', 'date_time', 'hour_of_day', 'is_weekend', 'year', 'month', 'day', 'month_name', 'day_name', 'week', 'year_month', 'CO2_lag_1', 'kWh_lag_1', 'CO2_lag_2', 'kWh_lag_2', 'CO2_lag_3', 'kWh_lag_3', 'CO2_lag_4', 'kWh_lag_4', 'temp_humid_interaction', 'log_kWh', 'log_humidity', 'day_of_week_Friday', 'day_of_week_Monday', 'day_of_week_Saturday', 'day_of_week_Sunday', 'day_of_week_Thursday', 'day_of_week_Tuesday', 'day_of_week_Wednesday']
CO2_imputed summary:
count    30365.000000
mean         0.026670
std          0.005963
min         

C:\Users\srsch\AppData\Local\Temp\ipykernel_26556\789275472.py:59: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  print(df.groupby('humidity_binned')['CO2_imputed'].var())
C:\Users\srsch\AppData\Local\Temp\ipykernel_26556\789275472.py:63: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  print(df.groupby('humidity_binned')['relative_humidity_2m (%)'].agg(['min', 'max']))
C:\Users\srsch\AppData\Local\Temp\ipykernel_26556\789275472.py:65: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=Tru

Updated dataset saved to C:/Users/srsch/OneDrive/Desktop/steel_weather_data_restored_v6.csv


In [18]:
import pandas as pd
import numpy as np

# Load source dataset
df = pd.read_csv(r'C:/Users/srsch/OneDrive/Desktop/merged_steel_weather_data_with_datetime.csv')
print("Source shape:", df.shape)

# Create humidity_binned
df['humidity_binned'] = pd.qcut(df['relative_humidity_2m (%)'], q=3, labels=['Low', 'Medium', 'High'])

# Impute CO2(tCO2) zeros with bin-specific medians
bin_medians = df[df['CO2(tCO2)'] != 0].groupby('humidity_binned', observed=False)['CO2(tCO2)'].median()
# Scale medians to match original
scaling_factors = {
    'Low': 0.028423 / bin_medians['Low'],
    'Medium': 0.026889 / bin_medians['Medium'],
    'High': 0.026414 / bin_medians['High']
}
df['CO2_imputed'] = df.groupby('humidity_binned', observed=False)['CO2(tCO2)'].transform(
    lambda x: x.replace(0, x[x != 0].median() * scaling_factors[x.name] if x[x != 0].size > 0 else np.nan)
)

# Stricter outlier removal (10th–90th percentiles)
co2_lower, co2_upper = df['CO2_imputed'].quantile([0.10, 0.90])
kwh_lower, kwh_upper = df['Usage_kWh'].quantile([0.10, 0.90])
df = df[
    (df['CO2_imputed'].between(co2_lower, co2_upper)) &
    (df['Usage_kWh'].between(kwh_lower, kwh_upper))
]

# Add time-based features
df['date_time'] = pd.to_datetime(df['date'])
df['hour_of_day'] = df['date_time'].dt.hour
df['is_weekend'] = df['Day_of_week'].isin(['Saturday', 'Sunday']).astype(int)
df['year'] = df['date_time'].dt.year
df['month'] = df['date_time'].dt.month
df['day'] = df['date_time'].dt.day
df['month_name'] = df['date_time'].dt.strftime('%B')
df['day_name'] = df['Day_of_week']
df['week'] = df['date_time'].dt.isocalendar().week
df['year_month'] = df['date_time'].dt.strftime('%Y-%m')

# Add lags
for lag in range(1, 5):
    df[f'CO2_lag_{lag}'] = df['CO2_imputed'].shift(lag)
    df[f'kWh_lag_{lag}'] = df['Usage_kWh'].shift(lag)

# Add interaction and transformations
df['temp_humid_interaction'] = df['temperature_2m (°C)'] * df['relative_humidity_2m (%)']
df['log_kWh'] = np.log1p(df['Usage_kWh'])
df['log_humidity'] = np.log1p(df['relative_humidity_2m (%)'])

# Add day_of_week dummies
df = pd.get_dummies(df, columns=['Day_of_week'], prefix='day_of_week')

# Drop rows with NaN
df = df.dropna()

# Verify
print("New shape:", df.shape)
print("Columns:", df.columns.tolist())
print("CO2_imputed summary:")
print(df['CO2_imputed'].describe())
print("CO2_imputed variance by humidity bin:")
print(df.groupby('humidity_binned', observed=False)['CO2_imputed'].var())
print("humidity_binned value counts:")
print(df['humidity_binned'].value_counts())
print("relative_humidity_2m (%) range by bin:")
print(df.groupby('humidity_binned', observed=False)['relative_humidity_2m (%)'].agg(['min', 'max']))
print("CO2_imputed medians:")
current_medians = df.groupby('humidity_binned', observed=False)['CO2_imputed'].median()
original_medians = {'Low': 0.028423, 'Medium': 0.026889, 'High': 0.026414}
print(pd.DataFrame({'Current': current_medians, 'Original': pd.Series(original_medians)}))

# Save updated dataset
output_csv = r'C:/Users/srsch/OneDrive/Desktop/steel_weather_data_restored_v7.csv'
df.to_csv(output_csv, index=False)
print(f"Updated dataset saved to {output_csv}")

Source shape: (35040, 20)
New shape: (26608, 49)
Columns: ['date', 'Usage_kWh', 'Lagging_Current_Reactive.Power_kVarh', 'Leading_Current_Reactive_Power_kVarh', 'CO2(tCO2)', 'Lagging_Current_Power_Factor', 'Leading_Current_Power_Factor', 'NSM', 'WeekStatus', 'Load_Type', 'date_only', 'time_only', 'time_numeric_dec', 'time_seconds_since_midnight', 'temperature_2m (°C)', 'relative_humidity_2m (%)', 'dew_point_2m (°C)', 'precipitation (mm)', 'Load_Type_Rank', 'humidity_binned', 'CO2_imputed', 'date_time', 'hour_of_day', 'is_weekend', 'year', 'month', 'day', 'month_name', 'day_name', 'week', 'year_month', 'CO2_lag_1', 'kWh_lag_1', 'CO2_lag_2', 'kWh_lag_2', 'CO2_lag_3', 'kWh_lag_3', 'CO2_lag_4', 'kWh_lag_4', 'temp_humid_interaction', 'log_kWh', 'log_humidity', 'day_of_week_Friday', 'day_of_week_Monday', 'day_of_week_Saturday', 'day_of_week_Sunday', 'day_of_week_Thursday', 'day_of_week_Tuesday', 'day_of_week_Wednesday']
CO2_imputed summary:
count    26608.000000
mean         0.026444
std     

In [19]:
from xgboost import XGBRegressor
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np

# Load dataset
df = pd.read_csv(r'C:/Users/srsch/OneDrive/Desktop/steel_weather_data_restored_v7.csv')
print("Dataset shape:", df.shape)
print("Low humidity rows:", len(df[df['humidity_binned'] == 'Low']))

# Define features (exclude log_kWh)
features = [
    'Usage_kWh', 'temperature_2m (°C)', 'relative_humidity_2m (%)', 'hour_of_day',
    'is_weekend', 'CO2_lag_1', 'temp_humid_interaction'
]
day_of_week_cols = [col for col in df.columns if col.startswith('day_of_week_')]
features.extend(day_of_week_cols)
print("Features used:", features)

# Low Humidity subset
df_low = df[df['humidity_binned'] == 'Low'].copy()
X_low = df_low[features]
y_low = df_low['CO2_imputed']

# Check feature statistics
print("Feature variances:", X_low.var())

# Scale features
scaler = StandardScaler()
X_low_scaled = scaler.fit_transform(X_low)
X_low_scaled = pd.DataFrame(X_low_scaled, columns=features, index=X_low.index)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_low_scaled, y_low, test_size=0.2, random_state=42)

# Hyperparameter tuning
param_grid = {
    'n_estimators': [700, 1000],
    'max_depth': [3, 4],
    'learning_rate': [0.005, 0.01],
    'min_child_weight': [1, 2],
    'subsample': [0.95, 1.0],
    'colsample_bytree': [0.95, 1.0]
}
model = XGBRegressor(random_state=42)
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='r2', n_jobs=-1)
grid_search.fit(X_train, y_train)
print("Best parameters:", grid_search.best_params_)
print("Best CV R²:", grid_search.best_score_)

# Train final model
model_low = grid_search.best_estimator_
model_low.fit(X_train, y_train)

# Evaluate
y_train_pred = model_low.predict(X_train)
y_test_pred = model_low.predict(X_test)
print("Low Humidity Train R²:", r2_score(y_train, y_train_pred))
print("Low Humidity Test R²:", r2_score(y_test, y_test_pred))
print("Low Humidity Train MSE:", mean_squared_error(y_train, y_train_pred))
print("Low Humidity Test MSE:", mean_squared_error(y_test, y_test_pred))

# Feature Importance
importances = pd.DataFrame({'Feature': features, 'Importance': model_low.feature_importances_})
print("Feature Importance:")
print(importances.sort_values('Importance', ascending=False))
importances.to_csv(r'C:/Users/srsch/OneDrive/Desktop/feature_importance_low_xgboost_v13.csv', index=False)


Dataset shape: (26608, 49)
Low humidity rows: 9214
Features used: ['Usage_kWh', 'temperature_2m (°C)', 'relative_humidity_2m (%)', 'hour_of_day', 'is_weekend', 'CO2_lag_1', 'temp_humid_interaction', 'day_of_week_Friday', 'day_of_week_Monday', 'day_of_week_Saturday', 'day_of_week_Sunday', 'day_of_week_Thursday', 'day_of_week_Tuesday', 'day_of_week_Wednesday']
Feature variances: Usage_kWh                      695.871500
temperature_2m (°C)            117.507486
relative_humidity_2m (%)       122.232102
hour_of_day                     34.667253
is_weekend                       0.217252
CO2_lag_1                        0.000015
temp_humid_interaction      343279.378482
day_of_week_Friday               0.141803
day_of_week_Monday               0.115526
day_of_week_Saturday             0.140586
day_of_week_Sunday               0.127354
day_of_week_Thursday             0.127430
day_of_week_Tuesday              0.097780
day_of_week_Wednesday            0.103319
dtype: float64
Best parameters: 

In [20]:
from xgboost import XGBRegressor
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np

# Load dataset
df = pd.read_csv(r'C:/Users/srsch/OneDrive/Desktop/steel_weather_data_restored_v7.csv')
print("Dataset shape:", df.shape)
print("Low humidity rows:", len(df[df['humidity_binned'] == 'Low']))

# Define features (exclude log_kWh)
features = [
    'Usage_kWh', 'temperature_2m (°C)', 'relative_humidity_2m (%)', 'hour_of_day',
    'is_weekend', 'CO2_lag_1', 'temp_humid_interaction'
]
day_of_week_cols = [col for col in df.columns if col.startswith('day_of_week_')]
features.extend(day_of_week_cols)
print("Features used:", features)

# Low Humidity subset
df_low = df[df['humidity_binned'] == 'Low'].copy()
X_low = df_low[features]
y_low = df_low['CO2_imputed']

# Check feature statistics
print("Feature variances:", X_low.var())

# Scale features
scaler = StandardScaler()
X_low_scaled = scaler.fit_transform(X_low)
X_low_scaled = pd.DataFrame(X_low_scaled, columns=features, index=X_low.index)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_low_scaled, y_low, test_size=0.2, random_state=42)

# Hyperparameter tuning
param_grid = {
    'n_estimators': [700, 1000],
    'max_depth': [3, 4],
    'learning_rate': [0.005, 0.01],
    'min_child_weight': [1, 2],
    'subsample': [0.95, 1.0],
    'colsample_bytree': [0.95, 1.0]
}
model = XGBRegressor(random_state=42)
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='r2', n_jobs=-1)
grid_search.fit(X_train, y_train)
print("Best parameters:", grid_search.best_params_)
print("Best CV R²:", grid_search.best_score_)

# Train final model
model_low = grid_search.best_estimator_
model_low.fit(X_train, y_train)

# Evaluate
y_train_pred = model_low.predict(X_train)
y_test_pred = model_low.predict(X_test)
print("Low Humidity Train R²:", r2_score(y_train, y_train_pred))
print("Low Humidity Test R²:", r2_score(y_test, y_test_pred))
print("Low Humidity Train MSE:", mean_squared_error(y_train, y_train_pred))
print("Low Humidity Test MSE:", mean_squared_error(y_test, y_test_pred))

# Feature Importance
importances = pd.DataFrame({'Feature': features, 'Importance': model_low.feature_importances_})
print("Feature Importance:")
print(importances.sort_values('Importance', ascending=False))
importances.to_csv(r'C:/Users/srsch/OneDrive/Desktop/feature_importance_low_xgboost_v13.csv', index=False)

Dataset shape: (26608, 49)
Low humidity rows: 9214
Features used: ['Usage_kWh', 'temperature_2m (°C)', 'relative_humidity_2m (%)', 'hour_of_day', 'is_weekend', 'CO2_lag_1', 'temp_humid_interaction', 'day_of_week_Friday', 'day_of_week_Monday', 'day_of_week_Saturday', 'day_of_week_Sunday', 'day_of_week_Thursday', 'day_of_week_Tuesday', 'day_of_week_Wednesday']
Feature variances: Usage_kWh                      695.871500
temperature_2m (°C)            117.507486
relative_humidity_2m (%)       122.232102
hour_of_day                     34.667253
is_weekend                       0.217252
CO2_lag_1                        0.000015
temp_humid_interaction      343279.378482
day_of_week_Friday               0.141803
day_of_week_Monday               0.115526
day_of_week_Saturday             0.140586
day_of_week_Sunday               0.127354
day_of_week_Thursday             0.127430
day_of_week_Tuesday              0.097780
day_of_week_Wednesday            0.103319
dtype: float64
Best parameters: 

In [22]:
import pandas as pd
import os

# Load final dataset
df = pd.read_csv(r"C:\Users\srsch\OneDrive\Desktop\merged_steel_weather_data_final.csv")
df["date"] = pd.to_datetime(df["date"])

# Downcast function
def downcast_dtypes(df):
    for col in df.select_dtypes(include=['float64']).columns:
        df[col] = df[col].astype('float32')
    for col in df.select_dtypes(include=['int64']).columns:
        df[col] = df[col].astype('int32')
    return df

# Recreate df_clean (example, adjust as needed)
df_clean = df.dropna(subset=['relative_humidity_2m (%)', 'temperature_2m (°C)', 'CO2(tCO2)'])
import numpy as np
humidity_bins = np.linspace(df_clean['relative_humidity_2m (%)'].min(), df_clean['relative_humidity_2m (%)'].max(), 6)
df_clean['humidity_bin'] = pd.cut(df_clean['relative_humidity_2m (%)'], bins=humidity_bins, include_lowest=True)
temp_bins = np.linspace(df_clean['temperature_2m (°C)'].min(), df_clean['temperature_2m (°C)'].max(), 4)
df_clean['temp_bin'] = pd.cut(df_clean['temperature_2m (°C)'], bins=temp_bins, include_lowest=True)
df_clean['humidity_bin'] = df_clean['humidity_bin'].astype(str)
df_clean['temp_bin'] = df_clean['temp_bin'].astype(str)
df_clean = downcast_dtypes(df_clean)
df_clean.to_parquet('df_clean.parquet', index=True)
print("Saved df_clean:", os.path.exists('df_clean.parquet'))

# Save other DataFrames (adjust based on your definitions)
df_no_outliers = downcast_dtypes(df_no_outliers)
df_no_outliers.to_parquet('df_no_outliers.parquet', index=True)
print("Saved df_no_outliers:", os.path.exists('df_no_outliers.parquet'))

df_nonzero = downcast_dtypes(df_nonzero)
df_nonzero.to_parquet('df_nonzero.parquet', index=True)
print("Saved df_nonzero:", os.path.exists('df_nonzero.parquet'))

df = downcast_dtypes(df)
df.to_parquet('df.parquet', index=True)
print("Saved df:", os.path.exists('df.parquet'))

# Install memory_profiler
%pip install memory_profiler
%load_ext memory_profiler
%memit

Saved df_clean: True
Saved df_no_outliers: True


NameError: name 'df_nonzero' is not defined